In [223]:
# this is our preamble cell :
# remember to check for anything missing 
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib 


import category_encoders as ce
from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

from sklearn import cluster
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree

In [57]:
# OK, importing and minor cleaning first. 

dfreal = pd.read_csv('True.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')
dfreal['Fake'] = 0
print('Real data shape: ', dfreal.shape)

dffake = pd.read_csv('Fake.csv',
                    parse_dates = ['date'])
#                    index_col = 'date')

dffake['Fake'] = 1
print('Fake data shape: ', dffake.shape)

Real data shape:  (21417, 5)
Fake data shape:  (23481, 5)


In [58]:
# this is proof that only the fake csv has garbage dates in it, which is why none of the parse dates worked. 
dfreal['date'].value_counts()
#.tail(50)

2017-12-20    182
2017-12-06    166
2017-11-30    162
2017-11-09    158
2017-10-13    155
             ... 
2016-09-11      1
2016-05-14      1
2016-05-30      1
2016-08-06      1
2016-09-03      1
Name: date, Length: 716, dtype: int64

In [59]:
dffake['date'].value_counts()

May 10, 2017                                                                                                                                             46
May 5, 2016                                                                                                                                              44
May 26, 2016                                                                                                                                             44
May 6, 2016                                                                                                                                              44
May 11, 2016                                                                                                                                             43
                                                                                                                                                         ..
November 20, 2017                                               

In [84]:

# dffake = 
comma_list = pd.DataFrame(dffake['date'].str.find(','))

comma_list.value_counts()
comma_list.shape

(23481, 1)

In [108]:
# df = df[~df['your column'].isin(['list of strings'])]
# http = ['http',".com"]
# dffake2 = dffake[~dffake.date.isin(http)]

searchfor = ['http', '-', 'MSNBC']
dffake2 = dffake[~dffake['date'].str.contains('|'.join(searchfor))]

In [109]:
dffake2['date'].value_counts()

May 10, 2017         46
May 26, 2016         44
May 6, 2016          44
May 5, 2016          44
May 11, 2016         43
                     ..
October 9, 2017       1
December 19, 2017     1
November 19, 2017     1
October 22, 2017      1
December 9, 2017      1
Name: date, Length: 1669, dtype: int64

In [110]:
print(dffake2.shape)
print(dfreal.shape)

(23436, 5)
(21417, 5)


In [105]:
#dffake2['date'] = pd.to_datetime(dffake2['date'], format='%m%d%y')
# dffake2['date'] = dffake2['date'].astype('datetime64[ns]')

In [116]:
# now I'll trim those up so they are the same length - 
# 50% real 50% fake seems reasonable right?

dfreal_trimmed = dfreal[-21_400 :]
print('Real trimmed shape: ', dfreal_trimmed.shape)

dffake_trimmed = dffake2[-21_400 :]
print('Fake trimmed shape: ', dffake_trimmed.shape)

# and now combine them into one dataframe:
df_joined = dfreal_trimmed.append(dffake_trimmed, ignore_index=True)

df_joined['date'] = pd.to_datetime(df_joined['date'])

print()
print('Combined and trimmed (equal parts Real and Fake) shape: ', df_joined.shape)
print()
#print(df_joined.head(1))
#df_joined.head(25)

Real trimmed shape:  (21400, 5)
Fake trimmed shape:  (21400, 5)

Combined and trimmed (equal parts Real and Fake) shape:  (42800, 5)



In [119]:
df_joined['date'].value_counts()

# Holy Smokes I think all the date times are clean. 
# Let's never spend 2 days on that again. 

2017-12-20    194
2017-12-06    180
2017-11-09    178
2017-11-30    175
2017-10-13    171
             ... 
2015-06-21      1
2015-06-07      1
2015-07-19      1
2015-07-18      1
2015-04-02      1
Name: date, Length: 1004, dtype: int64

In [12]:

# All URL's and Wrong-Dates seem to be corrected. Finally.  
# X and y are still not the same length - need to fix that... 



In [162]:
df_joined.head()

,title,text,subject,date,Fake
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",The following statements were posted to the ve...,politicsNews,2017-12-26,0
1,U.S. appeals court rejects challenge to Trump ...,(Reuters) - A U.S. appeals court in Washington...,politicsNews,2017-12-26,0
2,Treasury Secretary Mnuchin was sent gift-wrapp...,(Reuters) - A gift-wrapped package addressed t...,politicsNews,2017-12-24,0
3,Federal judge partially lifts Trump's latest r...,WASHINGTON (Reuters) - A federal judge in Seat...,politicsNews,2017-12-24,0
4,Exclusive: U.S. memo weakens guidelines for pr...,NEW YORK (Reuters) - The U.S. Justice Departme...,politicsNews,2017-12-23,0


## *I don't think we'll use this but here's a Count Vectorizer just in case:*

In [122]:
# instantiate the count vectorizer:
cv = CountVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 50, max_df = 0.95 , min_df = 0.01 )

# here's a function to return a dataframe:

def create_term_matrix(message_list, vectorizer):
    doc_term_df = vectorizer.fit_transform(message_list)
    return DataFrame(doc_term_df.toarray(),
                     columns=vectorizer.get_feature_names())

# now here's the actual "thing":

df_joined_CountVector = create_term_matrix(df_joined['text'], cv)

df_joined_CountVector.describe()

,according,american,called,campaign,clinton,country,court,democratic,did,donald,...,support,time,told,trump,united,washington,week,white,year,years
count,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,...,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000,42800.000000
mean,0.315047,0.360117,0.295794,0.483505,0.618014,0.389159,0.329299,0.303575,0.369673,0.590771,...,0.297734,0.460935,0.529416,2.523014,0.536028,0.422009,0.299813,0.483178,0.557126,0.386589
std,0.787854,1.065031,0.658973,1.347394,2.304309,0.904674,1.353023,0.888128,0.797977,1.020930,...,0.768017,0.910293,0.939773,4.386813,1.283105,0.954925,0.675320,1.305252,1.120097,0.890714
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,20.000000,48.000000,10.000000,31.000000,67.000000,26.000000,43.000000,18.000000,20.000000,32.000000,...,24.000000,28.000000,13.000000,73.000000,42.000000,35.000000,10.000000,36.000000,27.000000,61.000000


In [ ]:
# Tf-IDF Vectorizer are the same results are doing a Count Vectorizer followed by a Tf-IDF Transform. 
# GradientBoostingClassifier. Good thing we learned some params today. 
# 
# I still have questions regarding how to merge the results from this with my orginal df_joined(41,800 x 5) - so pin in that for now. 

# 
# create a dictionary for myself (ex: year and years same word) - (sisichen)
# find an NLP function that can cluster similiar words together - lookup common NLP functions - (sisichen)

# merging original frame and target vector with results so that I can train test split and fit model...
# Ngrams. They exist in the parameters for the vectorizer / model below... 

## *now for TF-IDF* - check for class weight = balanced

In [199]:
# alright, let's see if I can generate anything to show for myself re: TF-IDF Vectorizer... 

# instantiate:
tfidf = TfidfVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 50, min_df= 0.25 , max_df= 0.75)
# ngram_range=(1,2) - we're gonna play with the ngrams soon, be patient. 
def create_term_matrix(message_list, vectorizer):
    doc_term_df = vectorizer.fit_transform(message_list)
    return DataFrame(doc_term_df.toarray(),
                     columns=vectorizer.get_feature_names())

joined_tfid01 = create_term_matrix(df_joined['text'], tfidf)

joined_tfid01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42800 entries, 0 to 42799
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   did         42800 non-null  float64
 1   donald      42800 non-null  float64
 2   government  42800 non-null  float64
 3   house       42800 non-null  float64
 4   just        42800 non-null  float64
 5   like        42800 non-null  float64
 6   new         42800 non-null  float64
 7   news        42800 non-null  float64
 8   people      42800 non-null  float64
 9   president   42800 non-null  float64
 10  republican  42800 non-null  float64
 11  reuters     42800 non-null  float64
 12  said        42800 non-null  float64
 13  state       42800 non-null  float64
 14  states      42800 non-null  float64
 15  time        42800 non-null  float64
 16  told        42800 non-null  float64
 17  trump       42800 non-null  float64
 18  united      42800 non-null  float64
 19  washington  42800 non-nul

In [201]:
# ok an attempt to get it back into the original frame so we can train test split... 

#df_joined_tfidvect = df_joined 
#df_joined_tfidvect['tfidf'] 

#first drop text
df_joined_tfidvect = df_joined.drop(['text'], axis=1)

#fit it - sparse series
tfidf_fit = tfidf.fit_transform(df_joined['text'])
print(tfidf_fit.shape)

(42800, 22)


In [202]:
#dataframe (temporary, using pd.DataFrame and .toarray)
df1 = pd.DataFrame(tfidf_fit.toarray(), columns=tfidf.get_feature_names())


#concat (after drop) for resulting frame
df_joined_tfidvect = pd.concat([df_joined_tfidvect, df1], axis=1)

df_joined_tfidvect.shape

(42800, 27)

In [185]:
df_joined_tfidvect.head()

,title,subject,date,Fake,tfidf,did,donald,government,house,just,...,said,state,states,time,told,trump,united,washington,year,years
0,"Trump on Twitter (Dec 26) - Hillary Clinton, T...",politicsNews,2017-12-26,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.190346,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.432356,0.000000,0.524103,0.00000,0.000000,0.638174,0.000000
1,U.S. appeals court rejects challenge to Trump ...,politicsNews,2017-12-26,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.15181,0.123258,0.143941,0.0,0.000000,...,0.250194,0.271940,0.697583,0.000000,0.000000,0.113127,0.14763,0.291545,0.000000,0.000000
2,Treasury Secretary Mnuchin was sent gift-wrapp...,politicsNews,2017-12-24,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.222543,0.000000,0.0,0.000000,...,0.150575,0.000000,0.000000,0.000000,0.719777,0.408504,0.00000,0.000000,0.000000,0.000000
3,Federal judge partially lifts Trump's latest r...,politicsNews,2017-12-24,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.119426,0.139466,0.0,0.000000,...,0.323220,0.131743,0.405537,0.000000,0.000000,0.548050,0.42912,0.141240,0.133467,0.147699
4,Exclusive: U.S. memo weakens guidelines for pr...,politicsNews,2017-12-23,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.064380,0.075184,0.0,0.071809,...,0.479166,0.000000,0.072873,0.219353,0.000000,0.118178,0.07711,0.000000,0.071950,0.000000


In [217]:
#df_joined_tfidvect.drop(['title'], axis=1, inplace=True)

df_joined_tfidvect['date'] = df_joined_tfidvect['date'].dt.year

df_joined_tfidvect.head()

,subject,date,Fake,tfidf,did,donald,government,house,just,like,...,said,state,states,time,told,trump,united,washington,year,years
0,politicsNews,2017,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.190346,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.432356,0.000000,0.524103,0.00000,0.000000,0.638174,0.000000
1,politicsNews,2017,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.15181,0.123258,0.143941,0.0,0.000000,0.142809,...,0.250194,0.271940,0.697583,0.000000,0.000000,0.113127,0.14763,0.291545,0.000000,0.000000
2,politicsNews,2017,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.222543,0.000000,0.0,0.000000,0.000000,...,0.150575,0.000000,0.000000,0.000000,0.719777,0.408504,0.00000,0.000000,0.000000,0.000000
3,politicsNews,2017,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.119426,0.139466,0.0,0.000000,0.000000,...,0.323220,0.131743,0.405537,0.000000,0.000000,0.548050,0.42912,0.141240,0.133467,0.147699
4,politicsNews,2017,0,"(0, 20)\t0.6381743283166692\n (0, 6)\t0.201...",0.00000,0.064380,0.075184,0.0,0.071809,0.000000,...,0.479166,0.000000,0.072873,0.219353,0.000000,0.118178,0.07711,0.000000,0.071950,0.000000


In [218]:
# ok, that's new for us... let's gooooooo

# X and y split



target = df_joined_tfidvect['Fake']

X = df_joined_tfidvect.drop(['Fake'], axis=1)

y = target

print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (42800, 25)
y shape:  (42800,)


In [219]:
# this is silly but it's good form, so here's a baseline. 
print('Baseline :', df_joined['Fake'].value_counts().max()/len(df_joined['Fake']))

Baseline : 0.5


In [220]:
# train test split:

# leaving this code here in case I set up my target wrong. 
# X = df.drop(['target'],axis=1).values   # independant features
# y = df['target'].values					# dependant variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# doing 25/75 split and 42. 

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(24075, 25)
(24075,)
(8025, 25)
(8025,)
(10700, 25)
(10700,)


In [164]:
# # instantiate 2nd: much different mix max:
# tfidf2 = TfidfVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 100, min_df= 0.5 , max_df= 0.99)
# # ngram_range=(1,2) - we're gonna play with the ngrams soon, be patient. 
# def create_term_matrix(message_list, vectorizer):
#     doc_term_df = vectorizer.fit_transform(message_list)
#     return DataFrame(doc_term_df.toarray(),
#                      columns=vectorizer.get_feature_names())

# df_joined_tfidfvector2 = create_term_matrix(df_joined['text'], tfidf2)

# df_joined_tfidfvector2.describe()

In [197]:
# instantiate: once again, different min max. 

tfidf3 = TfidfVectorizer(stop_words = 'english', strip_accents ='ascii', max_features = 50, min_df= 0.2 , max_df= 0.4)
# ngram_range=(1,2) - we're gonna play with the ngrams soon, be patient. 
def create_term_matrix(message_list, vectorizer):
    doc_term_df = vectorizer.fit_transform(message_list)
    return DataFrame(doc_term_df.toarray(),
                     columns=vectorizer.get_feature_names())

df_joined_tfidfvector3 = create_term_matrix(df_joined['text'], tfidf3)

df_joined_tfidfvector3.head()


,according,american,called,campaign,country,did,donald,election,going,government,...,states,time,told,united,washington,way,week,white,year,years
0,0.000000,0.0,0.000000,0.498824,0.000000,0.000000,0.192121,0.000000,0.259713,0.000000,...,0.000000,0.436389,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.644126,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.143514,0.116522,0.301517,0.000000,0.136075,...,0.659463,0.000000,0.000000,0.139562,0.275613,0.000000,0.000000,0.0,0.000000,0.000000
2,0.331001,0.0,0.000000,0.000000,0.000000,0.000000,0.249475,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.806885,0.000000,0.000000,0.000000,0.327407,0.0,0.000000,0.000000
3,0.184967,0.0,0.364490,0.000000,0.348085,0.000000,0.139410,0.000000,0.000000,0.162803,...,0.473397,0.000000,0.000000,0.500926,0.164874,0.187637,0.000000,0.0,0.155800,0.172414
4,0.000000,0.0,0.090353,0.000000,0.086286,0.000000,0.069116,0.000000,0.093432,0.080714,...,0.078233,0.235488,0.000000,0.082782,0.000000,0.000000,0.000000,0.0,0.077242,0.000000


In [ ]:
# from pdpbox.pdp import pdp_isolate, pdp_plot
# that should be pip installed now. 
# also pip installed plotly. 


In [221]:
print(X_train.shape)
print(y_train.shape)

(24075, 25)
(24075,)


In [226]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24075 entries, 7784 to 6475
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject     24075 non-null  object 
 1   date        24075 non-null  int64  
 2   tfidf       24075 non-null  object 
 3   did         24075 non-null  float64
 4   donald      24075 non-null  float64
 5   government  24075 non-null  float64
 6   house       24075 non-null  float64
 7   just        24075 non-null  float64
 8   like        24075 non-null  float64
 9   new         24075 non-null  float64
 10  news        24075 non-null  float64
 11  people      24075 non-null  float64
 12  president   24075 non-null  float64
 13  republican  24075 non-null  float64
 14  reuters     24075 non-null  float64
 15  said        24075 non-null  float64
 16  state       24075 non-null  float64
 17  states      24075 non-null  float64
 18  time        24075 non-null  float64
 19  told        24075 non-n

In [227]:
# # Create pipeline
X_train.drop(['tfidf'], axis=1, inplace=True)

model1 = Pipeline([
    #('vectorizer', TfidfVectorizer(lowercase=True, stop_words = 'english', strip_accents ='ascii', max_features = 20, ngram_range=(1,2))),
    #('dim_red', TruncatedSVD(n_components=20, random_state=42)),
    ('encoder', OrdinalEncoder()),
    ('predictor', GradientBoostingClassifier(random_state=42))
])

# Fit model to training data
model1.fit(X_train, y_train); 



In [208]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Lets simplify : df_joined DF (80/20) split

train = df_joined.loc[:34240,:]
test = df_joined.loc[8560:,:]

# For testing 

y = train['Fake']
X = train['text']

# Instantiate vectorizer object
vect = TfidfVectorizer(stop_words='english')
clf = xgb.XGBClassifier() #estimator
clf2 = GradientBoostingClassifier(random_state=42)

# Define the Pipeline
pipe = Pipeline([('vect', vect), ('clf', clf)])

# super basic parameter list for tuning
parameters = {
    'vect__max_features': [500, 1000],
    'clf__n_estimators':[75, 125]}
# create the gridsearch
grid_search = GridSearchCV(pipe, parameters, cv= 3, n_jobs=4, scoring='accuracy', verbose=1)
grid_search.fit(X, y) # fit the model

# results
print(grid_search.best_params_)
print(grid_search.best_score_)
# lets see how it does agaisnt the test
pred = grid_search.predict(test['text'])


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[18:44:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'clf__n_estimators': 75, 'vect__max_features': 1000}
0.996524610322128


In [ ]:
# importances = c26_model_xgb_est10.named_steps['xgbregressor'].feature_importances_
# features = c26X_train.columns
# feat_imp = pd.Series(importances, index=features).sort_values()
# feat_imp.tail(10).plot(kind='barh')
# plt.xlabel('Gini Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importances for model_skbg');